In [3]:
import torch
import pandas as pd
import numpy as np

In [4]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset["train"].features

Found cached dataset imdb (/Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [5]:
from transformers import BertTokenizerFast

tokenizer: BertTokenizerFast = BertTokenizerFast.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
    )

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached processed dataset at /Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-e0a0342ae289143d.arrow
Loading cached processed dataset at /Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-fd8ddff947474c37.arrow
Loading cached processed dataset at /Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-2a73e6194285aadb.arrow
Loading cached shuffled indices for dataset at /Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-95a7ea67f59766e0.arrow
Loading cached shuffled indices for dataset at /Users/louiechou/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-db88639656d75d2e

<a id='trainer'></a>

In [7]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoConfig,
    BertForSequenceClassification,
)

config = AutoConfig.from_pretrained(
    "bert-base-uncased",
    # num_labels=2,
)
# model = AutoModelForSequenceClassification.from_config(config)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="bert_pretrained",
    evaluation_strategy="epoch",
    use_mps_device=torch.backends.mps.is_available(),
    optim="adamw_torch",
)

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.348700,0.236343,0.898000
2,0.187800,0.320808,0.922000
3,0.067700,0.406377,0.924000


TrainOutput(global_step=3750, training_loss=0.2256259084065755, metrics={'train_runtime': 1022.0878, 'train_samples_per_second': 29.352, 'train_steps_per_second': 3.669, 'total_flos': 7893331660800000.0, 'train_loss': 0.2256259084065755, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [11]:
from transformers import BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained(
    "bert_pretrained",
    output_attentions=True,
)

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.3858458995819092,
 'eval_accuracy': 0.9256,
 'eval_runtime': 293.7491,
 'eval_samples_per_second': 85.107,
 'eval_steps_per_second': 10.638,
 'epoch': 3.0}

In [13]:
from bertviz import head_view, model_view

input_text = "The sign of a good movie is that it can toy with our emotions"
inputs = tokenizer.encode(input_text, return_tensors="pt")

attention = model(inputs)[-1]

tokens = tokenizer.convert_ids_to_tokens(inputs[0])
model_view(attention, tokens)

<IPython.core.display.Javascript object>